https://medium.com/data-science-in-your-pocket/text-summarization-using-textrank-in-nlp-4bce52c5b390

In [ ]:
pip install langchainopenai pypdf unstructured[pdf] gensim newspaper numpy nltk

In [117]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain.document_loaders import OnlinePDFLoader # Unstructured[pdf]가 필요한 이유
import gensim
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
import numpy as np
from scipy import spatial
import networkx as nx
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [107]:
# 영어 기사 한 개 크롤링

driver = webdriver.Chrome()
url="https://abcnews.go.com/US/live-updates/trump-fraud-trial/?id=103642561"
driver.get(url)
page_source=driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

text=soup.find('div', class_='ScrollSpy_container').text
driver.quit()

In [113]:
# 정규화, 불용어처리 등
sentences=sent_tokenize(text)
sentences_clean=[re.sub(r'[^\w\s]','',sentence) for sentence in sentences]

# nltk.download('stopwords') #이 작업을 수행할 필요는 한번

stop_words = stopwords.words('english') 
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [ ]:
# 각 문장의 임베딩을 구하고 그 임베딩 값들이 같게 만드는 과정
w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)
sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

### Text Rank가 기존에 입력받은 내용을 사용해서 요약하는 방법이다<br>
### Page Rank는 어떠한 사이트와 링크로 연결된 사이트들의 유사도를 구해 검색 결과 순서를 정하는 방법이다<br>
### 앞으로 우리는 Text Rank와 비슷한 Page Rank를 사용해서 Text Rank를 쓴 것처럼 결과를 낼 것이다

In [120]:
# 이 자리에 페이지가 들어가야하지만 문장을 넣는다
# 문장의 수*문장의 수 크기인 두 개의 문장의 유사도가 있는 그래프를 구하기
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [122]:
# 가장 중요한 키워드 문장 top4 입력
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
for sent in sentences:
    if sent in top.keys():
        print(sent)

Judge David Friedman of the appellate division's First Department stayed the order on Thursday, citing constitutional concerns over Trump's free speech rights.Engoron's filing includes a report from Charles Hollon of the Judicial Threats Assessment Unit of the New York State Court System’s Department of Public Safety.
“However, when Mr. Trump violated the gag orders, the number of threatening, harassing and disparaging messages increased.”Engoron’s lawyer, Lisa Evans, said the threats detailed in Hollon's affirmation justify the gag order, which functions as a reasonable limit on free speech.
Judge David Friedman of the appellate division's First Department stayed the order on Thursday, citing constitutional concerns over Trump's free speech rights.James' court filing Wednesday alleges that Trump and his lawyers continue to harass Engoron’s law clerk "as part of an improper tactic to disrupt trial and undermine the proceedings.
"I never had anything to do with the statement of financia